In [4]:
from nltk.tag import StanfordNERTagger
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

import os

stan_ner_classifier = "../../lib/stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz"
stan_ner_jar = "../../lib/stanford-ner/stanford-ner-3.8.0.jar"
tagger = StanfordNERTagger(stan_ner_classifier, stan_ner_jar);

/usr/local/lib/python2.7/dist-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


[(u'Rami', u'PERSON'),
 (u'Eid', u'PERSON'),
 (u'is', u'O'),
 (u'studying', u'O'),
 (u'at', u'O'),
 (u'Stony', u'ORGANIZATION'),
 (u'Brook', u'ORGANIZATION'),
 (u'University', u'ORGANIZATION'),
 (u'in', u'O'),
 (u'NY', u'O')]

In [44]:
doc_dir = 'DataSet/content'
P_TAG = "PERSON"
O_TAG = "ORGANIZATION"
L_TAG = "LOCATION"

persons_data = dict()
locations_data =dict()
organizations_data =dict() 

file_num= 1
for f in os.listdir(doc_dir):
    print 'processing file '+str(file_num)
    file_num = file_num + 1
    word_file_path = os.path.join(doc_dir, f)
    persons =[]
    locations = []
    organizations = []
    with open(word_file_path, "rt") as word_file:
        for line in word_file:
            tags = tagger.tag(line.split())
            #print tags
            all_word_count = len(tags)
            for i in xrange(all_word_count):
                if tags[i][1] == P_TAG:
                    persons.append((tags[i][0], i))
                elif tags[i][1] == L_TAG:
                    locations.append((tags[i][0], i))
                elif tags[i][1] == O_TAG:
                    organizations.append((tags[i][0], i))
    persons_data[f] = persons
    locations_data[f] = locations
    organizations_data[f] = organizations
    print persons
        

processing file 1
[(u'Sebastian', 1), (u'JAIPUR', 2), (u'Khwaja', 23), (u'Moinuddin', 24), (u'Chishti', 25), (u'Devendra', 48), (u'Gupta', 49), (u'Gupta', 58), (u'Gupta', 94), (u'Kapil', 143), (u'Garg', 144), (u'Gupta', 174), (u'Shanti', 181), (u'Kumar', 182), (u'Dhariwal', 183), (u'Devendra', 199), (u'Gupta', 200), (u'Dhariwal', 236), (u'Garg', 296), (u'Gupta', 310), (u'Garg', 325), (u'Garg', 343), (u'Garg', 384), (u'Garg', 425), (u'Ramzan', 484), (u'Maulanas', 588)]
processing file 2
[(u'JAIPUR', 2), (u'Devendra', 44), (u'Gupta', 45), (u'Chandrashekhar', 54), (u'Abhinav', 72), (u'Bharat', 73), (u'Chandrashekhar', 121), (u'Devendra', 147), (u'Chandrashekhar', 149), (u'Pragnya', 168), (u'Thakur', 169), (u'Srikant', 171), (u'Purohit', 172)]
processing file 3
[(u'Anita', 0), (u'Joshua', 1), (u'Over', 2), (u'Khyber-Pukhtoonkhwa', 143)]
processing file 4
[(u'Dilip', 118), (u'Kumar', 119), (u'Mitra', 120), (u'Buddhadeb', 306), (u'Bhattacharjee', 307), (u'Mitra', 357)]
processing file 5
[(u'

In [60]:
import pickle
import operator

In [30]:


with open('saved_py_objs/persons_data.p', 'wb')  as f:
    pickle.dump(persons_data, f)

with open('saved_py_objs/locations_data.p', 'wb')  as f:
    pickle.dump(locations_data, f)
    
with open('saved_py_objs/organizations_data.p', 'wb')  as f:
    pickle.dump(organizations_data, f)
    

In [45]:
with open('saved_py_objs/persons_data.p', 'rb')  as f:
    persons_data = pickle.load(f)

with open('saved_py_objs/locations_data.p', 'rb')  as f:
    locations_data = pickle.load(f)
    
with open('saved_py_objs/organizations_data.p', 'rb')  as f:
    organizations_data = pickle.load(f)


In [70]:
# Rank Top 3 persons
top_persons = dict()
top_locations = dict()
top_organizations = dict()

for doc_name in persons_data.keys():
    person_list = persons_data[doc_name]
    num_persons = len(person_list)
    person_freq = dict()
    person_pos = dict()
    person_score = dict()
    
    # Get max position of the person
    N = 1;
    for i in xrange(num_persons):
        person = person_list[i][0]
        if person in person_freq:
            person_freq[person] =  person_freq[person]+1
        else:
            person_freq[person] = 1
            person_pos[person] = person_list[i][1]
            if person_list[i][1] > N:
                N = person_list[i][1]
    for person in person_freq.keys():
        score  = (1+ person_freq[person]/N) + ((1+ N - person_pos[person]))
        person_score[person] = math.log10(score)
    person_score = sorted(person_score.items(), key=operator.itemgetter(1))
    top_persons[doc_name] =  person_score
    
for doc_name in organizations_data.keys():
    organization_list = organizations_data[doc_name]
    num_organization = len(organization_list)
    organization_freq = dict()
    organization_pos = dict()
    organization_score = dict()
    
    # Get max position of the person
    N = 1;
    for i in xrange(num_organization):
        organization = organization_list[i][0]
        if organization in organization_freq:
            organization_freq[organization] =  organization_freq[organization]+1
        else:
            organization_freq[organization] = 1
            organization_pos[organization] = organization_list[i][1]
            if organization_list[i][1] > N:
                N = organization_list[i][1]
    for organization in organization_freq.keys():
        score  = (1+ organization_freq[organization]/N) + ((1+ N - organization_pos[organization]))
        organization_score[organization] = math.log10(score)
    organization_score = sorted(organization_score.items(), key=operator.itemgetter(1))
    top_organizations[doc_name] =  organization_score
    
for doc_name in locations_data.keys():
    location_list = locations_data[doc_name]
    
    num_locations = len(location_list)
    location_freq = dict()
    location_pos = dict()
    location_score = dict()
    
    # Get max position of the person
    N = 1;
    for i in xrange(num_locations):
        location = location_list[i][0]
        if location in location_freq:
            location_freq[location] =  location_freq[location]+1
        else:
            location_freq[location] = 1
            location_pos[location] = location_list[i][1]
            if location_list[i][1] > N:
                N = location_list[i][1]
    for location in location_freq.keys():
        score  = (1+ location_freq[location]/N) + ((1+ N - location_pos[location]))
        location_score[location] = math.log10(score)    
    location_score = sorted(location_score.items(), key=operator.itemgetter(1))
    top_locations[doc_name] =  location_score
    


with open('saved_py_objs/top_persons_data.p', 'wb')  as f:
    pickle.dump(top_persons, f)

with open('saved_py_objs/top_locations_data.p', 'wb')  as f:
    pickle.dump(top_locations, f)
    
with open('saved_py_objs/top_organizations_data.p', 'wb')  as f:
    pickle.dump(top_organizations, f)

In [71]:
for doc_name in top_locations.keys():
    print doc_name
    print top_persons[doc_name]
    print top_locations[doc_name]
    print top_organizations[doc_name]
    print '\n'

2010_4_6_st-398.txt
[(u'Suicide', 0.3010299956639812), (u'Joshua', 0.47712125471966244), (u'Anita', 0.6020599913279624)]
[(u'Waziristan', 0.3010299956639812), (u'North', 0.47712125471966244), (u'Peshawar', 2.290034611362518), (u'U.S.', 2.298853076409707), (u'ISLAMABAD', 2.3180633349627615)]
[(u'AFP', 0.3010299956639812), (u'Taliban', 0.8450980400142568), (u'ISI', 1.7993405494535817), (u'Embassy', 2.173186268412274), (u'U.S.', 2.1760912590556813)]


2010_4_1_st-439.txt
[(u'Buryatsky', 0.3010299956639812), (u'Said', 0.47712125471966244), (u'Putin', 1.9030899869919435), (u'Radyuhin', 2.481442628502305), (u'Vladimir', 2.482873583608754)]
[(u'Metro', 0.3010299956639812), (u'Moscow', 0.47712125471966244), (u'Chechnya', 1.568201724066995), (u'Dagestan', 2.173186268412274), (u'Kyzlyar', 2.1931245983544616), (u'Caucasus', 2.2278867046136734), (u'North', 2.230448921378274), (u'Russia', 2.2600713879850747), (u'MOSCOW', 2.278753600952829)]
[(u'TNT', 0.3010299956639812)]


2010_4_17_st-395.txt
[(u'